In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import glob
from scipy import stats
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>")) 
import warnings
warnings.filterwarnings('ignore')

In [3]:
# data1516 = pd.read_csv('~/Desktop/Capstone/ManualSelect1516.csv')
# data1516.set_index('SEQN',inplace=True)
# data1314 = pd.read_csv('~/Desktop/Capstone/ManualSelect1314.csv')
# data1314.set_index('SEQN',inplace=True)

In [37]:
data1314= pd.read_csv('~/Desktop/Capstone/all_features_original_1314.csv')

In [38]:
data1516= pd.read_csv('~/Desktop/Capstone/all_features_original_1516.csv')

In [51]:
def get_different_columns(datasetList):
    colList = {}
    for i in range(len(datasetList)):
        colList[i]=(set(datasetList[i].columns))
    common = colList[0]
    for s in colList.values():
        common = common&s
    special={}
    for i in range(len(colList)):
        special[i] = colList[i] - common
    return common, special

In [52]:
common , special = get_different_columns([data1314,data1516])

In [53]:
data = pd.concat([data1314,data1516],axis=0,join='inner')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11466 entries, 0 to 5707
Columns: 1333 entries, SEQN to LBDVXYLC.1
dtypes: float64(1256), int64(1), object(76)
memory usage: 116.7+ MB


In [54]:
data.set_index('SEQN',inplace=True)

In [55]:
#print unsat feature by percentage threshold(unsaturated%)
def print_unsat_features(dataset, threshold):
    num = dataset.shape[0]*threshold
    for col in dataset.columns:
        missing = features[col].isnull().sum()
        if missing>num:
            print(col, missing)
    return
#print_unsat_features(features, 0.6)

##### 1. Drop under-saturated data

In [56]:
#drop unsat feature by percentage threshold(unsaturated%)
def drop_unsat_features(dataset, threshold):
    droplist = []
    num = dataset.shape[0]*threshold
    for col in dataset.columns:
        missing = dataset[col].isnull().sum()
        if missing>num:
            droplist.append(col)
    sat_data=dataset.drop(droplist,axis=1)
    return sat_data
sat_data = drop_unsat_features(data, 0.6)
sat_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11466 entries, 73557 to 93702
Columns: 730 entries, ACD011A to LBDVXYLC.1
dtypes: float64(654), object(76)
memory usage: 63.9+ MB


##### 2. Data Imputation by mean for continuos, mode for dicrete variables

In [60]:
#threshold of unique values to categorize feature into discrete and continuous variable
def imputation(dataset, threshold):
    imputed_dataset = dataset.copy()
    for col in dataset:
        #discrete by mode
        if imputed_dataset[col].nunique()<=threshold:
            imputed_dataset[col] = imputed_dataset[col].fillna(imputed_dataset[col].mode().iloc[0])
        #non-numeric feature type by mode:
        elif imputed_dataset[col].dtype != 'float' and imputed_dataset[col].dtype != 'int':
            imputed_dataset[col] = imputed_dataset[col].fillna(imputed_dataset[col].mode().iloc[0])
        #continous by mean
        else:
            imputed_dataset[col] = imputed_dataset[col].fillna(imputed_dataset[col].mean())
    return imputed_dataset
imputed_data = imputation(sat_data,20)

In [61]:
imputed_data.to_csv('~/Desktop/Capstone/imputed_all_data_1316.csv')

In [62]:
imputed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11466 entries, 73557 to 93702
Columns: 730 entries, ACD011A to LBDVXYLC.1
dtypes: float64(654), object(76)
memory usage: 63.9+ MB
